<a href="https://colab.research.google.com/github/jorgenery/ufba-mestrado/blob/main/ic0004_trabalho_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import itertools
import numpy as np
import time
# Matriz de Distancias Cidades
# Fonte: https://www.melhoresrotas.com/tabela-de-distancias-entre-cidades/br-bahia/
distancias = [
    # Salvador, Feira de Santana, Itabuna, Juazeiro, Camaçari, Barreiras, Ilhéus, Lauro de Freitas, Teixeira de Freitas, Jequié, Porto Seguro, Alagoinhas, Simões Filho, Eunápolis, Paulo Afonso
    [0, 112, 433, 505, 46, 861, 449, 24, 806, 367, 707, 112, 25, 645, 457],  # Salvador
    [115, 0, 348, 394, 98, 748, 364, 110, 721, 254, 623, 75, 89, 560, 377],  # Feira de Santana
    [436, 348, 0, 741, 419, 942, 30, 431, 376, 187, 277, 396, 409, 215, 724],  # Itabuna
    [508, 393, 743, 0, 491, 935, 759, 503, 1_116, 645, 1_017, 445, 482, 955, 364],  # Juazeiro
    [50, 98, 419, 491, 0, 847, 435, 30, 792, 353, 693, 80, 24, 631, 425],  # Camaçari
    [863, 748, 942, 936, 846, 0, 957, 858, 1_185, 793, 1_086, 823, 837, 1_024, 1_124],  # Barreiras
    [452, 364, 30, 757, 435, 955, 0, 447, 405, 203, 307, 412, 426, 244, 740],  # Ilhéus
    [26, 108, 429, 501, 31, 857, 444, 0, 802, 363, 703, 108, 21, 641, 453],  # Lauro de Freitas
    [809, 721, 376, 1_114, 792, 1_185, 405, 804, 0, 560, 225, 769, 783, 163, 1_098],  # Teixeira de Freitas
    [369, 254, 187, 647, 352, 792, 203, 364, 560, 0, 461, 329, 343, 399, 630],  # Jequié
    [711, 623, 277, 1_016, 694, 1_087, 307, 706, 225, 462, 0, 671, 684, 63, 999],  # Porto Seguro
    [116, 76, 396, 446, 77, 824, 412, 105, 769, 330, 670, 0, 88, 608, 346],  # Alagoinhas
    [28, 89, 410, 482, 21, 838, 425, 22, 783, 344, 684, 88, 0, 622, 433],  # Simões Filho
    [648, 560, 214, 953, 631, 1_024, 244, 643, 163, 399, 62, 608, 622, 0, 936],  # Eunápolis
    [462, 378, 728, 367, 423, 1_124, 744, 451, 1_101, 630, 1_002, 347, 434, 940, 0]  # Paulo Afonso
    ]
cidades = [
    "Salvador", "Feira de Santana", "Itabuna", "Juazeiro", "Camaçari",
    "Barreiras", "Ilhéus", "Lauro de Freitas", "Teixeira de Freitas",
    "Jequié", "Porto Seguro", "Alagoinhas", "Simões Filho", "Eunápolis",
    "Paulo Afonso"
    ]
def get_submatrix(cidades_selecionadas, cidades, distancias):
    # Verificar se todas as cidades estão na lista
    for cidade in cidades_selecionadas:
        if cidade not in cidades:
            raise ValueError(f"A cidade '{cidade}' não está na lista de cidades.")
    # Obter os índices das cidades selecionadas
    indices = [cidades.index(cidade) for cidade in cidades_selecionadas]

    # Criar a submatriz
    submatriz = []
    for i in indices:
        linha = [distancias[i][j] for j in indices]
        submatriz.append(linha)

    return submatriz

## Implementação Algoritmos

### Algoritmo de Força Bruta

In [ ]:
# Algoritmo de Força Bruta
def gerar_permutacoes(arr):
    """Gera todas as permutações de um array."""
    if len(arr) == 0:
        return [[]]

    permutacoes = []
    for i in range(len(arr)):
        restante = arr[:i] + arr[i+1:]
        for p in gerar_permutacoes(restante):
            permutacoes.append([arr[i]] + p)

    return permutacoes

def forca_bruta(matriz_distancias, distancia_maxima):
    n = len(matriz_distancias)
    rota_minima = None
    distancia_minima = float('inf')

    # Gera todas as permutações possíveis das cidades (exceto a inicial)
    cidades = list(range(1, n))
    permutacoes = gerar_permutacoes(cidades)

    for perm in permutacoes:
        rota = (0,) + tuple(perm) + (0,)
        distancia = sum(matriz_distancias[rota[i]][rota[i+1]] for i in range(len(rota) - 1))

        if distancia < distancia_minima:
            distancia_minima = distancia
            rota_minima = rota
        if distancia > distancia_maxima:
            continue

    if distancia_minima > distancia_maxima:
        return None, None

    return rota_minima, distancia_minima

### Algoritmo do Vizinho Mais Próximo

In [ ]:
# Algoritmo do Vizinho Mais Próximo
def vizinho_mais_proximo(matriz_distancias, cidade_inicial=0, distancia_maxima=float('inf')):
    n = len(matriz_distancias)
    visitadas = [False] * n
    rota = [cidade_inicial]
    distancia_total = 0

    cidade_atual = cidade_inicial
    visitadas[cidade_atual] = True

    for _ in range(n - 1):
        # Encontra a próxima cidade com a menor distância, não visitada
        proxima_cidade = np.argmin([matriz_distancias[cidade_atual][j] if not visitadas[j] else float('inf') for j in range(n)])
        distancia_para_proxima = matriz_distancias[cidade_atual][proxima_cidade]

        # Verifica se adicionar a próxima cidade excede a distância máxima
        if distancia_total + distancia_para_proxima > distancia_maxima:
            return None, None

        distancia_total += distancia_para_proxima
        rota.append(proxima_cidade)
        visitadas[proxima_cidade] = True
        cidade_atual = proxima_cidade

    # Adiciona a distância de volta para a cidade inicial
    distancia_final = matriz_distancias[cidade_atual][cidade_inicial]

    if distancia_total + distancia_final > distancia_maxima:
        return None, None

    distancia_total += distancia_final
    rota.append(cidade_inicial)

    return rota, distancia_total

### Algoritmo Held Karp

In [ ]:
def held_karp(distances, max_distance):
    n = len(distances)
    C = {}

    for k in range(1, n):
        C[(1 << k, k)] = (distances[0][k], 0)

    for subset_size in range(2, n):
        for subset in itertools.combinations(range(1, n), subset_size):
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            for k in subset:
                prev_bits = bits & ~(1 << k)
                res = []
                for m in subset:
                    if m == 0 or m == k:
                        continue
                    res.append((C[(prev_bits, m)][0] + distances[m][k], m))
                C[(bits, k)] = min(res)

    bits = (2**n - 1) - 1
    res = []
    routes = []
    for k in range(1, n):
        res.append((C[(bits, k)][0] + distances[k][0], k))
        routes.append(C[(bits, k)][1])
    opt, parent = min(res)

    return routes,opt

### Colonia de Formigas

In [ ]:
# Colônia de Formigas
def colonia_formigas(dist_matrix, max_distance):
  # todo implementar
  return [],0

### Valida certificado

In [ ]:
# Valida certificado
def calculate_distance(route, distances):
    return sum(distances[route[i]][route[i+1]] for i in range(len(route)-1)) + distances[route[-1]][route[0]]

### Comparação de Eficiência

In [ ]:
# Comparação de Eficiência
def compare_algorithms(cities, dist_matrix, max_distance):

    matriz_comparacao = []

    # Algoritmo de Força Bruta
    start_time = time.time()
    rota, distancia = forca_bruta(dist_matrix, max_distance)
    tempo = time.time() - start_time
    dados = ["Algoritmo Força bruta", distancia, tempo, rota ]
    matriz_comparacao.append(dados)

    # Algoritmo do Vizinho Mais Próximo
    start_time = time.time()
    rota, distancia = vizinho_mais_proximo(dist_matrix,cidade_inicial=0,distancia_maxima=max_distance)
    tempo = time.time() - start_time
    dados = ["Algoritmo do Vizinho Mais Próximo", distancia, tempo, rota ]
    matriz_comparacao.append(dados)

    # Algoritmo held_karp
    start_time = time.time()
    rota, distancia = held_karp(dist_matrix, max_distance)
    tempo = time.time() - start_time
    dados = ["Algoritmo held_karp", distancia, tempo, rota ]
    matriz_comparacao.append(dados)

    # Algoritmo Colônia de Formigas
    rota, distancia = colonia_formigas(dist_matrix, max_distance)
    tempo = time.time() - start_time
    dados = ["Algoritmo Colônia de Formigas", distancia, tempo, rota ]
    matriz_comparacao.append(dados)

    # Resultados
    for d in matriz_comparacao:
      if d[1] == None or d[1]==0:
        print(f"{d[0]}: Nenhuma resposta valida")
      else:
        print(f"{d[0]}: Distância = {d[1]} km, Tempo = {d[2]:.4f} s, Rota = {d[3]}")
    return matriz_comparacao

## Experimentos

In [ ]:

roteiros = [
    ["Salvador", "Feira de Santana", "Itabuna"],
    ["Salvador", "Feira de Santana", "Itabuna", "Juazeiro", "Camaçari"],
    ["Salvador", "Feira de Santana", "Itabuna", "Juazeiro", "Barreiras","Ilhéus"],
#    ["Salvador", "Feira de Santana", "Itabuna", "Ilhéus", "Lauro de Freitas", "Teixeira de Freitas"],
#    ["Salvador", "Feira de Santana", "Itabuna", "Juazeiro", "Barreiras", "Ilhéus", "Lauro de Freitas", "Teixeira de Freitas"],
#    ["Salvador", "Feira de Santana", "Itabuna", "Juazeiro", "Camaçari",
#    "Barreiras", "Ilhéus", "Lauro de Freitas", "Teixeira de Freitas",
#    "Jequié", "Porto Seguro", "Alagoinhas", "Simões Filho", "Eunápolis",
#    "Paulo Afonso"]
]
max_distance = 1800
matriz_resultados = []
for roteiro in roteiros:
  matriz_roteiro = get_submatrix(roteiro, cidades, distancias)
  # Comparar os algoritmos
  matriz_resultados.append(compare_algorithms(roteiro, matriz_roteiro, max_distance))

Algoritmo Força bruta: Distância = 896 km, Tempo = 0.0000 s, Rota = (0, 1, 2, 0)
Algoritmo do Vizinho Mais Próximo: Distância = 896 km, Tempo = 0.0001 s, Rota = [0, 1, 2, 0]
Algoritmo held_karp: Distância = 896 km, Tempo = 0.0000 s, Rota = [2, 1]
Algoritmo Colônia de Formigas: Nenhuma resposta valida
Algoritmo Força bruta: Distância = 1714 km, Tempo = 0.0002 s, Rota = (0, 4, 2, 3, 1, 0)
Algoritmo do Vizinho Mais Próximo: Distância = 1741 km, Tempo = 0.0001 s, Rota = [0, 4, 1, 2, 3, 0]
Algoritmo held_karp: Distância = 1714 km, Tempo = 0.0001 s, Rota = [3, 1, 1, 1]
Algoritmo Colônia de Formigas: Nenhuma resposta valida
Algoritmo Força bruta: Nenhuma resposta valida
Algoritmo do Vizinho Mais Próximo: Nenhuma resposta valida
Algoritmo held_karp: Distância = 2862 km, Tempo = 0.0003 s, Rota = [3, 5, 4, 3, 2]
Algoritmo Colônia de Formigas: Nenhuma resposta valida


In [ ]:
matriz_resultados[0]

[['Algoritmo Força bruta', 896, 2.193450927734375e-05, (0, 1, 2, 0)],
 ['Algoritmo do Vizinho Mais Próximo',
  896,
  2.193450927734375e-05,
  [0, 1, 2, 0]]]